In [1]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()
import re

from bs4 import BeautifulSoup
import requests

/home/droman/Documents/stuff/dl_course/venv/lib/python3.8/site-packages/tqdm/std.py:670: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [9]:
response = requests.get("https://en.wikipedia.org/wiki/List_of_conglomerates")

In [10]:
html_tree = BeautifulSoup(response.content, 'html.parser')

In [11]:
li_list = html_tree.findAll('li')

In [20]:
def filter_condition(x):
    try:
        return x.find('a').get('title') is not None and x.find('a').get('href') is not None
    except:
        return False

li_list = list(filter(filter_condition, li_list))

In [30]:
result = []

In [31]:
def scrab_companies_from_list(links_list, result):
    error_count = 0
    for link in tqdm(links_list):
        try:
            title = link.find('a').get('title')
            href = link.find('a').get('href')
            company_page = requests.get(f"https://en.wikipedia.org/{href}")
            company_page_html_tree = BeautifulSoup(company_page.content, 'html.parser')

            result.extend(
                get_relation_from_first_paragraph(company_page_html_tree, title)
            )
            get_relations(company_page_html_tree, title, result)

        except:
            error_count+=1
            print("error")
    print(f"Error count: {error_count}")
    df = pd.DataFrame(result)
    df.to_csv("wiki_data.csv", index=False, header=['name_1','name_2'])

In [32]:
scrab_companies_from_list(li_list, result)

100%|██████████| 1046/1046 [10:23<00:00,  1.68it/s]


Error count: 0


In [33]:
len(result)

6011

In [ ]:
example_company = li_list[3]

In [17]:
title = example_company.find('a').get('title')
href = example_company.find('a').get('href')
company_page = requests.get(f"https://en.wikipedia.org/{href}")

In [9]:
company_page_html_tree = BeautifulSoup(company_page.content, 'html.parser')

In [19]:
def get_relation_from_first_paragraph(company_page_html_tree, title):
    final_list = []
    try:
        b_list = company_page_html_tree.find('p').findAll('b')
        if b_list is not None:
            for b in b_list:
                b = b.text
                if b != title:
                    final_list.append([title, b])
    except:
        pass
    return final_list

In [11]:
get_relation_from_first_paragraph(company_page_html_tree, title)

[['Nation Media Group', 'NMG']]

In [18]:
def _get_relations(company_page_html_tree, title, final_list, regexp):
    subsidary_companies = company_page_html_tree.find('span', id=lambda x: x and re.match(regexp, x))
    if subsidary_companies != None:
        if subsidary_companies.find_next_sibling('ul') is None:
            subsidary_companies = subsidary_companies.parent
        ul = subsidary_companies.find_next_sibling('ul')
        if ul is not None:
            companies_list = ul.findChildren('li')
            companies_list = list(map(lambda x: [title, x.text], companies_list))
            final_list.extend(companies_list)
        else:
            companies_list = subsidary_companies.find_next_siblings('dl')
            if companies_list is not None:
                companies_list = list(map(lambda x: [title, x.text], companies_list))
                final_list.extend(companies_list)

    return final_list

def get_relations(company_page_html_tree, title, final_list):
    regexp_list = [r'.*(Subsidiar|subsidiar).*', r'.*(Divisions|divisions).*', r'List of developments', r'.*(Business|business).*',
                    r'.*(Associated|associated).*',r'.*(Financial|financial).*', r'.*(Affiliates|affiliates).*', r'Group companies',

                   r'Shipping',r'Logistics',r'Finance', r'.*(Manufacturing|manufacturing).*', r'Shipping Services', r'Social Services',
                   r'Internet & Business', r'Joint ventures and foreign brands', r'Engineering',

                   r'Related organizations', r'Core members', r'Other members', r'Oil and gas', r'Real estate', r'Other business',  r'Other core businesses',

                   r'Japan', r'East Asia',r'Europe', r'North America', r'Oceania', r'South America', r'South Asia', r'Southeast Asia',r'Regional',
                   r'Long distance and international',

                   r'Mobile', r'.*(data|Data).*', r'Information security', r'Soft drink business', r'Alcoholic beverage business', r'Alcohol business',
                   r'Health', r'Food', r'Agribio', r'Nutrient food', r'Restaurants', r'Pharmaceutical business', r'Agribio business'

                   r'Electronics.*',r'Chemical.*',r'Telecommunications.*',r'Utilities.*',r'Banking.*', r'Foods.*', r'.*(Infrastructure|infrastructure).*',
                   r'Divestment', r'Motion Pictures', r'Television Production.*', r'Media.*']
    for regexp in regexp_list:
        _get_relations(company_page_html_tree, title, final_list, regexp)

get_relations(company_page_html_tree, title, [])

In [3]:
company_page = requests.get("https://en.wikipedia.org/wiki/Nass_Corporation")
company_page_html_tree = BeautifulSoup(company_page.content, 'html.parser')

In [7]:
tag = company_page_html_tree.find('span', id=lambda x: x and re.match(r'(Divisions|divisions).*', x))
tag = tag.parent

tag = tag.find_next_siblings('dl')
# companies_list = tag.findChildren('li')

companies_list = list(map(lambda x: x.text, tag))
# companies_list
companies_list

['Nass Industrial Services (NIS)',
 'Nass Scafform Contracting',
 'Nass Sand Processing Plant (NSPP)',
 'Nass Commercial Services',
 'Nass Ice Plant',
 'Nass Foods',
 'Nass Dredging Company',
 'Nass Contracting Co. W.L.L.',
 'Nass Electrical Contracting Co. W.L.L.',
 'Delmon Ready Mixed Concrete and Products Co. W.L.L. (DRMC)',
 'Delmon Precast Co. W.L.L. (DPC)']

In [27]:
test = [[1, 2], [3,4]]
h = [["qwe", "rty"]]
test = h + test
test = pd.DataFrame(test)
test.to_csv("test1.csv", index=False, header=["q", "w"])

In [2]:
wiki_data = pd.read_csv("./wiki_data.csv")

In [4]:
wiki_data

,name_1,name_2
0,Cevital,Cevital Food Processing Industry
1,Cevital,Cevital Food Processing Industry
2,Cevital,Cevital Food Processing Industry
3,Kenya Commercial Bank Group,KCB Group Limited
4,Kenya Commercial Bank Group,KCB Group
...,...,...
6006,Printable version of this page [p],Andritz AG
6007,Printable version of this page [p],Red Bull GmbH
6008,Printable version of this page [p],Voestalpine AG
6009,Printable version of this page [p],Infratil


In [39]:
wiki_data = wiki_data.drop_duplicates()

In [40]:
wiki_data = wiki_data.dropna()

In [41]:
# qqq = list(filter(lambda x_: type(x_[1]) != str, wiki_data.values.tolist()))
# list(map(lambda x1: x1[1], qqq))

[]

399

In [120]:
filtered_data = list(filter(lambda x_: len(x_[1].split()) <= 5 and re.search(r":", x_[1]) is None, wiki_data.values.tolist()))
len(list(map(lambda x1: x1[1], filtered_data)))

2976

In [121]:
def clear(pair):
    def _clear(company):
        company = re.sub("\d+", "", company)
        company = re.sub("\(.*\)|\[.*\]", "", company)
        company = re.sub("\n+", "", company)
        return company
    return [_clear(pair[0]), _clear(pair[1])]
cleared_data = list(map(clear, filtered_data))

In [125]:
cleared_data_df = pd.DataFrame(cleared_data, columns=['name_1', 'name_2'])
cleared_data_df = cleared_data_df.drop_duplicates()
cleared_data_df.to_csv('./cleared_wiki_data.csv', index=False)